In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
from scipy.cluster.vq import vq, kmeans, whiten
import lbg 


In [4]:
#Divid images into blocks
#data:array
#n:size of block
def blocking(data:np.array,n):
    print(np.shape(data))
    width= np.shape(data)[1]
    height= np.shape(data)[0]
    BlockVec=[]
    for y in range(0, height, n):
        for x in range(0, width, n):
            block = data[y:y + n, x:x + n]
            vector = block.flatten()
            BlockVec.append(vector)
    return BlockVec

#Blocking images in folder
#Ipath:path of folder
#return:[[image1_blocks],[image2_blocks],...]
def imageBlock(Ipath,n):
    folder_path=Ipath
    result=[]
    files = os.listdir(folder_path)
    for file in files:
        if os.path.isfile(os.path.join(folder_path, file)) and file[len(file)-3:]=="raw":
            print(file[len(file)-3:])
            print(os.path.join(folder_path, file))
            # Define image dimensions
            width = 256
            height = 256
            # Read the raw image data
            with open(os.path.join(folder_path, file), 'rb') as f:
                # Read the binary data into a NumPy array
                # 8-bit
                img_data = np.fromfile(f,dtype=np.uint8)
                # Reshape the 1D array to a 2D image
                img = img_data.reshape((height, width))
                result.append(blocking(img,n))
    return result

def entropy(data):
    hist, _ = np.histogram(data)
    total_count = len(data)
    # print(total_count)
    # print(hist)
    probabilities = hist / total_count
    # print(probabilities)
    probabilities=probabilities[probabilities!=0]
    # print(probabilities)
    entropy=-np.sum(probabilities * np.log2(probabilities))
    return entropy
    

In [6]:

imgBlock=imageBlock("training",8)
for i in range(len(imgBlock)):
    whitened = whiten(imgBlock[i])
    codebook,distortion=kmeans(whitened,k_or_guess=64)
    codebookIdex,indiviDistor=vq(whitened,codebook)
    print(codebookIdex)
    print("entropy:",entropy(codebookIdex))
    print("distortioin:",np.mean(indiviDistor))
    

raw
training/people.raw
(256, 256)
raw
training/boat.raw
(256, 256)
raw
training/bike.raw
(256, 256)
[52  8  8 ... 62 62 54]
entropy: 3.2619028519441384
distortioin: 1.028108401603883
[12 17 17 ... 21  1  9]
entropy: 3.2303247033234035
distortioin: 1.8788787099939277
[50 50 50 ... 10 10 10]
entropy: 3.2343045924481615
distortioin: 3.8887389909663197


(5)Using huffman encoding to further compress the sequence

In [7]:
imgBlock=imageBlock("training",4)
whitened = whiten(imgBlock[0])
codebook,distortion=kmeans(whitened,k_or_guess=64)
codebookIdex,indiviDistor=vq(whitened,codebook)
print(codebookIdex)
print("entropy:",entropy(codebookIdex))
print("distortioin:",np.mean(indiviDistor))
    

raw
training/people.raw
(256, 256)
raw
training/boat.raw
(256, 256)
raw
training/bike.raw
(256, 256)
[15 21 21 ... 56 20 20]
entropy: 3.134859047066062
distortioin: 0.37382666556163757


In [17]:

# Save binary data to a file
with open("codebookIdex.txt", 'wb') as Ofile:
    codebookIdex.tofile(Ofile)


codebookOut was the output of my previous implementation of huffman coding.

huffmancoding_global.cpp

In [22]:

with open("codebookOut", 'r') as f:
    furthercompress=np.fromfile(f,dtype=np.int8)
    print(furthercompress)
    print("entropy",entropy(furthercompress))
    

[ 11 -31  28 ... -76 -19 -16]
entropy 3.1045736610944883


Tree-Structured Vector Quantization

In [10]:
imgBlock=imageBlock("training",8)
for i in range(len(imgBlock)):
    whitened = whiten(imgBlock[i])
    for cb_size in (8,32,64):
        print('generating codebook for size', cb_size)
        cb, cb_abs_w, cb_rel_w = lbg.generate_codebook(whitened, cb_size)
        print(cb)
        codebookIdex,indiviDistor=vq(whitened,cb)
        print(codebookIdex)
        print("entropy:",entropy(codebookIdex))
        print("distortioin:",np.mean(indiviDistor))
    
    


raw
training/people.raw
(256, 256)
raw
training/boat.raw
(256, 256)
raw
training/bike.raw
(256, 256)
generating codebook for size 8
[[3.041986158213312, 3.024215244702825, 2.9126014647132723, 2.9554335855328437, 2.9633744656685397, 2.959882701664762, 2.9830320249079914, 3.001471840816466, 3.0312807250977505, 3.0300610052421986, 2.8974137483656843, 2.936759490485963, 2.960572847663153, 2.9678707621666995, 2.9865332593005665, 3.0047238157339016, 2.997829217343238, 3.017266943470872, 2.8850351766436466, 2.9154773652816566, 2.934511235172009, 2.947845012597579, 2.9679468193101357, 2.9737749412540806, 3.0055212005444316, 3.0111374494120717, 2.8957356064411095, 2.9287880190428193, 2.934921681658223, 2.9575557720293904, 2.96609984053221, 2.974343112106803, 2.9935033510462254, 3.016444596198196, 2.8985876439330642, 2.942017865095424, 2.9445166727503858, 2.966033232775031, 2.9683643852078596, 2.9621405858468135, 3.005206647287702, 3.0018028185594634, 2.8928552297053125, 2.944344587285424, 2.949